# ArcFace + Swin
- dataset, label, arcface parameter, modify
- train, val, test, dfinitely split
- W&B logging
- pretrain = False, learning rates
- bounding
- ArcMarginProduct learned
- Search Result
- Aug

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts

import timm
from torchvision import datasets, transforms
import time
from utils.utils import GaussianBlur , GeM
import math
import random
import numpy as np



from pytorch_metric_learning import losses, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


In [2]:
import os

os.environ["WANDB_NOTEBOOK_NAME"] = "ArcFace-DPat-SwinV2"

In [3]:
import wandb
wandb.login()
wandb.init(project="ArcFace-Swin",
           notes="EffNetb5_Arc_0326, augmentation with 800/768 resolution, random flip",
           name="EffNetb5_Arc_0326_800768"
           )


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find ArcFace-DPat-SwinV2.
wandb: Currently logged in as: min1000 (min1k). Use `wandb login --relogin` to force relogin


In [4]:

gem_pool = GeM()

class PatentNet(nn.Module):
    def __init__(self, model_name: str, pretrained: bool, embedding_size: int, in_features):
        super().__init__()
        

        self.backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=0, in_chans=1)

        self.pool = gem_pool
        self.head = nn.Sequential(

            nn.Flatten(),

            nn.Linear(in_features, embedding_size),
            nn.BatchNorm1d(embedding_size),
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)

        return x

In [5]:
app_count = 33364 

device = torch.device("cuda")


model = PatentNet(model_name='efficientnet_b5.sw_in12k_ft_in1k', pretrained="True", embedding_size=512 , in_features = 2048).to(device)

model = nn.DataParallel(model)

In [7]:
batch_size = 128

log_interval = 5

num_epochs = 15

In [8]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
scaler = GradScaler()
def train(model, metric, loss_func, device, train_loader, optimizer, metric_optimizer, epoch, scheduler = None, metric_scheduler = None):
    print(f"Epoch {epoch} training start")
    model.train()
    total_loss = 0
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device) # labels
        optimizer.zero_grad()
        metric_optimizer.zero_grad()

        with autocast():
            output = model(data)
            output = metric(output,labels)
            
            loss = loss_func(output, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.step(metric_optimizer)
        scaler.update()

        total_loss += loss

        if batch_idx % 330 == 0:
            print("Epoch {} Iteration {}: Train Loss = {}".format(epoch, batch_idx, loss))
        if batch_idx % log_interval == 0:
            wandb.log({'epoch': epoch, 'train-iter': batch_idx, 'train-loss': loss})
        
        wandb.log({'lr': optimizer.param_groups[0]['lr'], 'metric_lr': metric_optimizer.param_groups[0]['lr']})
        
    if scheduler:
        scheduler.step()
        metric_scheduler.step()

    total_loss = total_loss/len(train_loader)
    wandb.log({'epoch': epoch, 'train-epoch-loss':total_loss})
    print("End of epoch {}".format(epoch))

In [9]:
### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

In [10]:
### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    torch.cuda.empty_cache()
    model.eval()
    with torch.no_grad():
        train_embeddings, train_labels = get_all_embeddings(train_set, model)
        test_embeddings, test_labels = get_all_embeddings(test_set, model)
        train_labels = train_labels.squeeze(1)
        # print(train_labels)
        test_labels = test_labels.squeeze(1)
        print("Computing accuracy")
        accuracies = accuracy_calculator.get_accuracy(
            test_embeddings, test_labels, train_embeddings,  train_labels, False
        )
        prec_1 = accuracies["precision_at_1"]
        map = accuracies["mean_average_precision"]
        print(f"Test set accuracy (Precision@1) = {prec_1}, (mAP) = {map}")
        wandb.log({'Prec@1': accuracies["precision_at_1"], 'mAP': accuracies["mean_average_precision"]})


In [11]:
# test loss
def test_calc(model, metric, loss_func, device, test_query_loader, epoch):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for batch_idx, (data, labels) in enumerate(test_query_loader):
            data, labels = data.to(device), labels.to(device) # labels
            embeddings = model(data)
            output = metric(embeddings,labels)
            loss = loss_func(output, labels)
            total_loss += loss
            
            if batch_idx % 50 == 0:
                print("Epoch {} Iteration {}: Test Loss = {}".format(epoch, batch_idx, loss))
            if batch_idx % log_interval == 0:
                wandb.log({'epoch': epoch, 'test-iter': batch_idx, 'test-loss': loss})
    
        total_loss = total_loss/len(test_query_loader)
        wandb.log({'epoch': epoch, 'test-epoch-loss':total_loss})
    

In [12]:
### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def valuation(val_db_set, val_query_set, model, accuracy_calculator):
    torch.cuda.empty_cache()
    model.eval()
    with torch.no_grad():
        val_db_embeddings, val_db_labels = get_all_embeddings(val_db_set, model)
        val_query_embeddings, val_query_labels = get_all_embeddings(val_query_set, model)
        val_db_labels = val_db_labels.squeeze(1)
        val_query_labels = val_query_labels.squeeze(1)
        print("Val accuracy")
        accuracies = accuracy_calculator.get_accuracy(
            val_query_embeddings, val_query_labels,val_db_embeddings, val_db_labels, False
        )
        prec_1 = accuracies["precision_at_1"]
        map = accuracies["mean_average_precision"]
        print(f"Val set accuracy (Precision@1) = {prec_1}, (mAP) = {map}")
        wandb.log({'Val_Prec@1': accuracies["precision_at_1"], 'Val_mAP': accuracies["mean_average_precision"]})

In [13]:

def valuation_calc(model, metric, loss_func, device, val_query_loader, epoch):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for batch_idx, (data, labels) in enumerate(val_query_loader):
            data, labels = data.to(device), labels.to(device) # labels
            embeddings = model(data)
            output = metric(embeddings,labels)
            loss = loss_func(output, labels)
            total_loss += loss
            
            if batch_idx % 50 == 0:
                print("Epoch {} Iteration {}: Test Loss = {}".format(epoch, batch_idx, loss))
            if batch_idx % log_interval == 0:
                wandb.log({'epoch': epoch, 'val-iter': batch_idx, 'validation-loss': loss})
        
        total_loss = total_loss/len(val_query_loader)
        wandb.log({'epoch': epoch, 'validation-epoch-loss':total_loss})

## transform, dataset, dataloader

In [14]:

data_transform = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize(size=(800,800)), 
        # transforms.RandomCrop((384,384), pad_if_needed=True, fill=255),
        transforms.CenterCrop((768,768)),
        transforms.RandomHorizontalFlip(p=0.5),
        # transforms.RandomApply([GaussianBlur()], p=0.5),
        transforms.ToTensor(),
        # transforms.RandomErasing(p=0.5, scale=(0.22, 0.33), ratio=(0.3, 3.3), value=1, inplace=False), 
        # transforms.Normalize((0.5), (0.5)),
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize(size=(800,800)),
        transforms.CenterCrop((768,768)),
        transforms.ToTensor(),
        # transforms.Normalize((0.5), (0.5)),
    ])
}

## Dataset

In [15]:
import os.path as osp
import glob
def make_datapath_list(phase):

    
    rootpath = "/work/v24684491/DeepPatent/patent_data/"


    if phase == "train":
        with open('./patlist/train_patent_trn.txt') as f:
            lines = f.readlines()
    elif phase == "test_query":
        with open('./patlist/test_query_patent.txt') as f:
            lines = f.readlines()
    elif phase == "test_db":
        with open('./patlist/test_db_patent.txt') as f:
            lines = f.readlines()
    elif phase == "val_query":
        with open('./patlist/val_query_patent.txt') as f:
            lines = f.readlines()
    elif phase == "val_db":
        with open('./patlist/val_db_patent.txt') as f:
            lines = f.readlines()
            
    path_list = [] 
    for path in lines:
        path = path.split()[0] # 0:path only, 1: label
        path_list.append(rootpath+path)
        
    return path_list

train_list = make_datapath_list(phase="train")

test_query_list = make_datapath_list(phase="test_query")
test_db_list = make_datapath_list(phase="test_db")

val_query_list = make_datapath_list(phase="val_query")
val_db_list = make_datapath_list(phase="val_db")


In [16]:
def make_label_list(phase):

    
    rootpath = "/home/hmc/DeepPatent/patent_data/"   # Enter the path to the dataset


    if phase == "train":
        with open('./patlist/train_patent_trn.txt') as f:
            lines = f.readlines()
    elif phase == "test_query":
        with open('./patlist/test_query_patent.txt') as f:
            lines = f.readlines()
    elif phase == "test_db":
        with open('./patlist/test_db_patent.txt') as f:
            lines = f.readlines()
    elif phase == "val_query":
        with open('./patlist/val_query_patent.txt') as f:
            lines = f.readlines()
    elif phase == "val_db":
        with open('./patlist/val_db_patent.txt') as f:
            lines = f.readlines()
        
    label_list = [] 
    for label in lines:
        label = label.split()[1] # 0:path only, 1: label
        label = int(label)
        label_list.append(label)
        
    return label_list

train_label_list = make_label_list(phase="train")

test_query_label_list = make_label_list(phase="test_query")
test_db_label_list = make_label_list(phase="test_db")

val_query_label_list = make_label_list(phase="val_query")
val_db_label_list = make_label_list(phase="val_db")



In [17]:
import copy
import torch.utils.data as data
from PIL import Image
import cv2

def cv2pil(image):

    new_image = image.copy()
    if new_image.ndim == 2:  
        pass
    elif new_image.shape[2] == 3:  
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
    elif new_image.shape[2] == 4:  
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGRA2RGBA)
    new_image = Image.fromarray(new_image)
    return new_image

class PatentDataset(data.Dataset):
    def __init__(self, file_list, label_list, transform):
        self.file_list = file_list
        self.label_list = label_list
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # open
        _, img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)  

        contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
        cnt = contours[0]

        x, y, w, h = cv2.boundingRect(cnt) 
        img = img[y:y + h, x:x + w]  



        img = cv2pil(img)
        img = self.transform(img)

        label = self.label_list[index]
        label = label - 1  # 0-indexed [0, nclass-1]

        return img, label


In [18]:
import os

num_workers = 16
print(num_workers)

16


In [19]:
train_data = PatentDataset(file_list=train_list, label_list=train_label_list, transform=data_transform['train'])

test_query_data = PatentDataset(file_list=test_query_list, label_list=test_query_label_list, transform=data_transform['val'])
test_db_data = PatentDataset(file_list=test_db_list, label_list=test_db_label_list, transform=data_transform['val'])

val_query_data = PatentDataset(file_list=val_query_list, label_list=val_query_label_list, transform=data_transform['val'])
val_db_data = PatentDataset(file_list=val_db_list, label_list=val_db_label_list, transform=data_transform['val'])

In [20]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)

test_query_loader = torch.utils.data.DataLoader(test_query_data, batch_size=batch_size, num_workers=num_workers, drop_last=True)
test_db_loader = torch.utils.data.DataLoader(test_db_data, batch_size=batch_size, num_workers=num_workers, drop_last=True)

val_query_loader = torch.utils.data.DataLoader(val_query_data, batch_size=batch_size, num_workers=num_workers, drop_last=True)
val_db_loader = torch.utils.data.DataLoader(val_db_data, batch_size=batch_size, num_workers=num_workers, drop_last=True)

## Setting


In [21]:
from pytorch_metric_learning import losses, distances, regularizers



## ArcFace

In [22]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=64.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        # print(output)

        return output

## loss, optimizer, accuracy setting

In [23]:

optimizer = optim.Adam(model.parameters(), lr=1e-4)
metric = ArcMarginProduct(512, app_count, s=20.0, m=0.50, easy_margin=True).to(device)
metric_optimizer = optim.Adam(metric.parameters(), lr=5e-3)


scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs )
metric_scheduler = CosineAnnealingLR(metric_optimizer, T_max=num_epochs )



In [24]:

class FocalLoss(nn.Module):

    def __init__(self, gamma=1.5, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss()

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [25]:

loss_func = FocalLoss()

# loss_optimizer = optim.Adam(loss_func.parameters(), lr=1e-5) # from 1e-4


accuracy_calculator = AccuracyCalculator(include=("precision_at_1", "mean_average_precision"), k="max_bin_count")
### pytorch-metric-learning stuff ###

In [26]:
import os
import torch

file_name = "/work/v24684491/Saved_models/240326_EffNetb5_Arc_800768/"
if not os.path.exists(file_name):
    os.makedirs(file_name)


## Train the model

In [27]:


for epoch in range(1, num_epochs + 1):
    
    time_start = time.time()
#     train(model, loss_func, device, train_loader, optimizer, loss_optimizer, epoch)
    train(model, metric, loss_func, device, train_loader, optimizer, metric_optimizer, epoch, scheduler, metric_scheduler)
#     train(model, metric, loss_func, device, train_loader, optimizer, metric_optimizer, epoch)
    
    torch.save({'state_dict': model.module.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            "metric_optimizer_dict" : metric_optimizer.state_dict(),
            "metric_dict" : metric.state_dict(),
            "lr_scheduler_dict" : scheduler.state_dict(),
            "metric_lr_scheduler_dict" : metric_scheduler.state_dict(),
            'epochs' : epoch }, file_name +'Swinv2_Arc{}.pth'.format(epoch))
    
    # torch.save({"state_dict":model.state_dict(),"epoch":epoch}, file_name +'EffArc_Gem-test_epoch{}.pth'.format(epoch))
    
    time_end = time.time()   
    time_c= time_end - time_start   
    print(f'Epoch {epoch} training time cost {round(time_c/60,2)} mins') 
    
    valuation(val_db_data, val_query_data, model, accuracy_calculator)
    # valuation_calc(model, metric, loss_func, device, val_query_loader, epoch)

    val_time_end = time.time()    
    time_c= val_time_end - time_end   
    print(f'Epoch {epoch} val time cost {round(time_c/60,2)} mins')
    
    if epoch == num_epochs:
        test(test_db_data, test_query_data, model, accuracy_calculator)
        # test_calc(model, metric, loss_func, device, test_query_loader, epoch)

        test_time_end = time.time()    
        time_c= test_time_end - val_time_end   
        print(f'Epoch {epoch} test time cost {round(time_c/60,2)} mins')

wandb.finish()

Epoch 1 training start
Epoch 1 Iteration 0: Train Loss = 16.085472106933594
Epoch 1 Iteration 330: Train Loss = 16.093978881835938
Epoch 1 Iteration 660: Train Loss = 13.462297439575195
Epoch 1 Iteration 990: Train Loss = 13.14050006866455
Epoch 1 Iteration 1320: Train Loss = 11.898300170898438
Epoch 1 Iteration 1650: Train Loss = 10.906122207641602
Epoch 1 Iteration 1980: Train Loss = 9.137556076049805
End of epoch 1
Epoch 1 training time cost 37.71 mins


100%|██████████| 353/353 [02:57<00:00,  1.99it/s]


Val accuracy


/home/v24684491/.local/lib/python3.10/site-packages/faiss/contrib/torch_utils.py:51: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  x.storage().data_ptr() + x.storage_offset() * 4)


Val set accuracy (Precision@1) = 0.9312394641114363, (mAP) = 0.8064360313449676
Epoch 1 val time cost 12.12 mins
Epoch 2 training start
Epoch 2 Iteration 0: Train Loss = 7.656277179718018
Epoch 2 Iteration 330: Train Loss = 7.708886623382568
Epoch 2 Iteration 660: Train Loss = 7.918742656707764
Epoch 2 Iteration 990: Train Loss = 7.749048709869385
Epoch 2 Iteration 1320: Train Loss = 7.752127170562744
Epoch 2 Iteration 1650: Train Loss = 6.998295783996582
Epoch 2 Iteration 1980: Train Loss = 6.605541229248047
End of epoch 2
Epoch 2 training time cost 37.49 mins


...

100%|██████████| 353/353 [03:39<00:00,  1.60it/s]


Val accuracy
Val set accuracy (Precision@1) = 0.9695679176648034, (mAP) = 0.9121211905455285
Epoch 15 val time cost 14.51 mins


100%|██████████| 411/411 [04:23<00:00,  1.56it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.9646691540394426, (mAP) = 0.9034256129829507
Epoch 15 test time cost 17.21 mins


Prec@1,▁
Val_Prec@1,▁▅▆▇▇▇█████████
Val_mAP,▁▅▆▇███████████
epoch,▁▁▁▁▁▁▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇███
lr,████████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
mAP,▁
metric_lr,████████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
train-epoch-loss,█▅▄▃▃▃▂▂▂▁▁▁▁▁▁
train-iter,▂▄▇▃▅█▄▆▁▅▇▂▆█▄▇▂▅▇▃▆▁▃▇▂▄█▃▆▂▄▇▂▅▁▃▆▂▄█
train-loss,█▇▆▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
Prec@1,0.96467
